# import the pagereads file in C*

In [1]:
%%cql create KEYSPACE if not exists databricks WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1};

In [2]:
%%cql create table databricks.pagecounts (project text,
                                          title text,
                                          requests int,
                                          size bigint,
                                          primary key(project,title));

In [4]:
import org.apache.spark.sql.types._

In [5]:
val schema = StructType(Seq(
    StructField("project", StringType, true),
    StructField("title", StringType, true),
    StructField("requests", IntegerType, true),
    StructField("size", LongType, true)))

In [6]:
val rawFile = sqlContext.read.format("com.databricks.spark.csv")
  .option("header", "false")
  .option("delimiter", " ")
  .option("mode", "DROPMALFORMED")
  .schema(schema)
  .load("file:///mnt/ephemeral/summitdata/pagecounts-20160210-180000")

In [7]:
rawFile.printSchema

root
 |-- project: string (nullable = true)
 |-- title: string (nullable = true)
 |-- requests: integer (nullable = true)
 |-- size: long (nullable = true)



In [8]:
rawFile.first

org.apache.spark.sql.Row = [aa,Category:Translators_deu-epo,1,4792]

In [ ]:
rawFile.write.format("org.apache.spark.sql.cassandra").
options(Map("keyspace" -> "databricks", "table" -> "pagecounts" )).
save()